# A people assignment problem.
## Copyright (C) Princeton Consultants, 2017

- Assign hours of work of people to sites per day of week
- Each person can work
  - At most a specified number of hours per day
  - At most a specified number of hours per week
  - At most a specified number of days per week
- Each site needs a set of people assigned for a number of hours per day
- When a person is assigned to a site
  - They must be assigned for at least an hour, in half-hour increments
  - They must be assigned for at least 2 days in the week
- There is a preference score for each person/site pair that is eligible to be assigned
  - Maximize the preferences
- There are not enough people available to cover all the site requirements, so maximize the coverage as best as possible 


## The list of files:
* sites.csv   contains site data:  ID and num hours per week
* people.csv  contains people data:  ID, total hours per week, days per week
* peoplehoursperday.csv contains hours per day for each person:  ID, day, hours
* sitehoursperday.csv contains hours per day for each site:  ID, day, hours
* prefs.csv contains the allowed assignments and a preference score:  person_id, site_id, pref


In [1]:
import pandas as pd
sites = pd.read_csv("sites.csv", index_col=0)
people = pd.read_csv("people.csv", index_col=0)
peoplehoursperday = pd.read_csv("peoplehoursperday.csv", index_col=[0,1])
sitehoursperday = pd.read_csv("sitehoursperday.csv", index_col=[0,1])
prefs = pd.read_csv("prefs.csv", index_col=[0,1])

In [2]:
sites.head()

,num_hours
id,
TWD0000000357BVYIM,12.0
UXE0000000358CWZJN,179.0
VYF0000000359DXAKO,171.0
WZG0000000360EYBLP,11.0
XAH0000000361FZCMQ,10.0


In [3]:
people.head()

,total_hrs_per_week,days_per_week
id,,
ADK0000000000ICFPT,6,3
BEL0000000001JDGQU,12,3
CFM0000000002KEHRV,10,7
DGN0000000003LFISW,28,6
EHO0000000004MGJTX,14,1


In [4]:
sitehoursperday.head()

hours
id                 day       
ADK0000000364ICFPT Mo     8.0
                   Tu     8.0
                   We     8.0
                   Th     8.0
                   Fr     4.0

In [5]:
peoplehoursperday.head()

,,hours
id,day,
ADK0000000000ICFPT,Mo,0
BEL0000000001JDGQU,Mo,8
CFM0000000002KEHRV,Mo,0
DGN0000000003LFISW,Mo,0
EHO0000000004MGJTX,Mo,0


In [6]:
prefs.head()

,,pref
person_id,site_id,
MPW0000000272UORBF,ADK0000000364ICFPT,64
ORY0000000196WQTDH,ADK0000000364ICFPT,32
QTA0000000146YSVFJ,ADK0000000364ICFPT,97
TWD0000000123BVYIM,ADK0000000364ICFPT,57
ADK0000000000ICFPT,ADK0000000390ICFPT,35


## Data checks.  Do people and peoplehoursperday correspond?  Compare the indexes

In [7]:
uniquePeople = set(people.index.get_level_values('id'))
print ("PeopleHoursPerDay and People refer to same people? ", 
       set(peoplehoursperday.index.get_level_values('id')) == uniquePeople)
print ("Number of people ", len(people))

PeopleHoursPerDay and People refer to same people?  True
Number of people  357


## Check that the people in the prefs table are in the people table (and see if some people never are). 


In [8]:
peopleInPrefs = set(prefs.index.get_level_values('person_id'))
print ("People in Prefs and People refer to same people? ", 
       (len(uniquePeople) == len(peopleInPrefs)) and
       (uniquePeople == peopleInPrefs))
print ("Number of people in prefs = ", len(peopleInPrefs))

People in Prefs and People refer to same people?  False
Number of people in prefs =  338


## This means that some people can never be assigned.  Now check that the people in the prefs table are in the people table

In [9]:
print ("People in prefs are in people table? ",
       peopleInPrefs.issubset(uniquePeople))


People in prefs are in people table?  True


## Time to check the sites.  Do same checks as with people, but in reverse

In [10]:
uniqueSites = set(sites.index.get_level_values('id'))
uniqueSitesPerDay = set(sitehoursperday.index.get_level_values('id'))
print ("SiteHoursPerDay and Sites refer to same sites? ",
       (len(uniqueSites) == len(uniqueSitesPerDay)) and
       uniqueSitesPerDay == uniqueSites)
print ("Number of sites ", len(uniqueSites))
print ("Number of sites from Hours Per day ", len(uniqueSitesPerDay))

SiteHoursPerDay and Sites refer to same sites?  False
Number of sites  548
Number of sites from Hours Per day  498


## So there are some sites that have no openings on any days.  Let's make a list of them

In [11]:
sitesWithNoDays = uniqueSites.difference(uniqueSitesPerDay)
print("Number of sites with no daily hours = ", len(sitesWithNoDays))

Number of sites with no daily hours =  50


## So we have seen that there are 50 sites that have no days they can be assigned.  Now check the sites against the prefs table

In [12]:
sitesInPrefs = set(prefs.index.get_level_values('site_id'))
print("Number of sites from prefs = ", len(sitesInPrefs))

Number of sites from prefs =  498


## Take a guess that the data is consistent in this regard

# The next step is to take the prefs table (eligible pairs), merge with the peoplehoursperday and the sitehoursperday, and determine how many hours can possibly be assigned to each pair (the min of the two values)

In [13]:
prefswithdaily = (
    pd.merge(prefs.reset_index(), peoplehoursperday.reset_index(), left_on='person_id', right_on='id', how='left')
    .drop('id',axis=1)
    .rename(columns={'hours' : 'person_hours'})
    .merge(sitehoursperday.reset_index(), left_on=['site_id','day'], right_on=['id','day'], how='left')
    .drop('id',axis=1)
    .rename(columns={'hours' : 'site_hours'})
    )
prefswithdaily.dropna(axis='index', how='any', subset=['site_hours'], inplace=True)
prefswithdaily['daily_hours'] = prefswithdaily[['person_hours','site_hours']].min(axis=1)
print("Length of prefs with daily, including zeros = ", len(prefswithdaily))
prefswithdaily = prefswithdaily.drop(['person_hours','site_hours'],axis=1)[prefswithdaily.daily_hours > 0]
print("Length of prefs with daily, without zeros = ",len(prefswithdaily))
prefswithdaily.head(20)

Length of prefs with daily, including zeros =  112364
Length of prefs with daily, without zeros =  87621


,person_id,site_id,pref,day,daily_hours
0,MPW0000000272UORBF,ADK0000000364ICFPT,64,Mo,8.0
1,MPW0000000272UORBF,ADK0000000364ICFPT,64,Tu,8.0
2,MPW0000000272UORBF,ADK0000000364ICFPT,64,We,8.0
3,MPW0000000272UORBF,ADK0000000364ICFPT,64,Th,4.0
4,MPW0000000272UORBF,ADK0000000364ICFPT,64,Fr,4.0
9,ORY0000000196WQTDH,ADK0000000364ICFPT,32,We,4.0
10,ORY0000000196WQTDH,ADK0000000364ICFPT,32,Th,4.0
11,ORY0000000196WQTDH,ADK0000000364ICFPT,32,Fr,4.0
14,QTA0000000146YSVFJ,ADK0000000364ICFPT,97,Mo,6.0
15,QTA0000000146YSVFJ,ADK0000000364ICFPT,97,Tu,6.0


In [14]:
from gurobipy import *
gurobipy.gurobi.version()

(7, 0, 2)

In [15]:
allvars = prefswithdaily.set_index(['person_id','site_id','day'])
allvars.head()

pref  daily_hours
person_id          site_id            day                   
MPW0000000272UORBF ADK0000000364ICFPT Mo     64          8.0
                                      Tu     64          8.0
                                      We     64          8.0
                                      Th     64          4.0
                                      Fr     64          4.0

In [16]:
eligiblePairs = set((p,s) for (p,s,d) in allvars.index)
eligiblePeople = set(p for p,s in eligiblePairs)
eligibleSites = set(s for p,s in eligiblePairs)
print (len(eligiblePairs), len(eligiblePeople), len(eligibleSites))

31427 337 498


In [17]:
m=Model("sched")

In [18]:
allvars['h'] = m.addVars(allvars.index, 
                         ub=2*allvars.daily_hours.values,
                         vtype=GRB.INTEGER,
                         name="h").values()
allvars['pairday'] = m.addVars(allvars.index,
                               vtype=GRB.BINARY,
                               name='pairday').values()
m.update()

In [19]:
pd.set_option('display.max_colwidth', 1000)
allvars.head(10)

pref  daily_hours  \
person_id          site_id            day                      
MPW0000000272UORBF ADK0000000364ICFPT Mo     64          8.0   
                                      Tu     64          8.0   
                                      We     64          8.0   
                                      Th     64          4.0   
                                      Fr     64          4.0   
ORY0000000196WQTDH ADK0000000364ICFPT We     32          4.0   
                                      Th     32          4.0   
                                      Fr     32          4.0   
QTA0000000146YSVFJ ADK0000000364ICFPT Mo     97          6.0   
                                      Tu     97          6.0   

                                                                                                  h  \
person_id          site_id            day                                                             
MPW0000000272UORBF ADK0000000364ICFPT Mo   <gurobi.Var h[MPW0000000272UORBF,ADK0000000364ICFPT,Mo]>   
                                      Tu   <gurobi.Var h[MPW0000000272UORBF,ADK0000000364ICFPT,Tu]>   
                                      We   <gurobi.Var h[MPW0000000272UORBF,ADK0000000364ICFPT,We]>   
                                      Th   <gurobi.Var h[MPW0000000272UORBF,ADK0000000364ICFPT,Th]>   
                                      Fr   <gurobi.Var h[MPW0000000272UORBF,ADK0000000364ICFPT,Fr]>   
ORY0000000196WQTDH ADK0000000364ICFPT We   <gurobi.Var h[ORY0000000196WQTDH,ADK0000000364ICFPT,We]>   
                                      Th   <gurobi.Var h[ORY0000000196WQTDH,ADK0000000364ICFPT,Th]>   
                                      Fr   <gurobi.Var h[ORY0000000196WQTDH,ADK0000000364ICFPT,Fr]>   
QTA0000000146YSVFJ ADK0000000364ICFPT Mo   <gurobi.Var h[QTA0000000146YSVFJ,ADK0000000364ICFPT,Mo]>   
                                      Tu   <gurobi.Var h[QTA0000000146YSVFJ,ADK0000000364ICFPT,Tu]>   

                                                                                                  pairday  
person_id          site_id            day                                                                  
MPW0000000272UORBF ADK0000000364ICFPT Mo   <gurobi.Var pairday[MPW0000000272UORBF,ADK0000000364ICFPT,Mo]>  
                                      Tu   <gurobi.Var pairday[MPW0000000272UORBF,ADK0000000364ICFPT,Tu]>  
                                      We   <gurobi.Var pairday[MPW0000000272UORBF,ADK0000000364ICFPT,We]>  
                                      Th   <gurobi.Var pairday[MPW0000000272UORBF,ADK0000000364ICFPT,Th]>  
                                      Fr   <gurobi.Var pairday[MPW0000000272UORBF,ADK0000000364ICFPT,Fr]>  
ORY0000000196WQTDH ADK0000000364ICFPT We   <gurobi.Var pairday[ORY0000000196WQTDH,ADK0000000364ICFPT,We]>  
                                      Th   <gurobi.Var pairday[ORY0000000196WQTDH,ADK0000000364ICFPT,Th]>  
                                      Fr   <gurobi.Var pairday[ORY0000000196WQTDH,ADK0000000364ICFPT,Fr]>  
QTA0000000146YSVFJ ADK0000000364ICFPT Mo   <gurobi.Var pairday[QTA0000000146YSVFJ,ADK0000000364ICFPT,Mo]>  
                                      Tu   <gurobi.Var pairday[QTA0000000146YSVFJ,ADK0000000364ICFPT,Tu]>

In [20]:
zvars = pd.DataFrame({'z' : m.addVars(eligiblePairs, vtype=GRB.BINARY, name="z").values()}, 
                     index=pd.MultiIndex.from_tuples(eligiblePairs, names=['person_id','site_id']))
m.update()
zvars.head()

,,z
person_id,site_id,
MPW0000000142UORBF,ADK0000000650ICFPT,"<gurobi.Var z[MPW0000000142UORBF,ADK0000000650ICFPT]>"
CFM0000000028KEHRV,UXE0000000904CWZJN,"<gurobi.Var z[CFM0000000028KEHRV,UXE0000000904CWZJN]>"
UXE0000000020CWZJN,PSZ0000000431XRUEI,"<gurobi.Var z[UXE0000000020CWZJN,PSZ0000000431XRUEI]>"
YBI0000000258GADNR,LOV0000000739TNQAE,"<gurobi.Var z[YBI0000000258GADNR,LOV0000000739TNQAE]>"
NQX0000000117VPSCG,BEL0000000391JDGQU,"<gurobi.Var z[NQX0000000117VPSCG,BEL0000000391JDGQU]>"


In [21]:
siteSlackVars = pd.DataFrame(
    { 'shortage' : m.addVars(eligibleSites, vtype=GRB.CONTINUOUS, name='shortage').values(),
      'excess' : m.addVars(eligibleSites, vtype=GRB.CONTINUOUS, name='excess').values() },
    index=pd.Index(eligibleSites, name='site_id'))
m.update()
print("Number of variables ", m.NumVars)
siteSlackVars.head()


Number of variables  207665


,excess,shortage
site_id,,
LOV0000000843TNQAE,<gurobi.Var excess[LOV0000000843TNQAE]>,<gurobi.Var shortage[LOV0000000843TNQAE]>
ADK0000000364ICFPT,<gurobi.Var excess[ADK0000000364ICFPT]>,<gurobi.Var shortage[ADK0000000364ICFPT]>
TWD0000000773BVYIM,<gurobi.Var excess[TWD0000000773BVYIM]>,<gurobi.Var shortage[TWD0000000773BVYIM]>
RUB0000000875ZTWGK,<gurobi.Var excess[RUB0000000875ZTWGK]>,<gurobi.Var shortage[RUB0000000875ZTWGK]>
SVC0000000512AUXHL,<gurobi.Var excess[SVC0000000512AUXHL]>,<gurobi.Var shortage[SVC0000000512AUXHL]>


## Define the objective function

In [22]:
m.setObjective(quicksum(siteSlackVars.shortage), sense=GRB.MINIMIZE)


## Define a helper function to name variables and constraints

In [23]:

import collections
def lpnamer(prefix, tup):
    if isinstance(tup, str):
        result = prefix + '[' + tup + ']'
    elif not isinstance(tup, collections.Iterable):
        result = prefix + '[' + str(tup) + ']'
    else:
        tup2 = [val for val in tup]
        tmp = '[' + ','.join(tup2) + ']'
        result = prefix + tmp
    return result

## Now time to do the constraints.  This version will mimic the OPL model

In [24]:
print (lpnamer('h','abc'), lpnamer('h',('abc','def')), lpnamer('h',('abc','def','ghi')))
for it in allvars.head(1).itertuples():
    print(type(it.Index))

h[abc] h[abc,def] h[abc,def,ghi]
<class 'tuple'>


In [25]:

conDF = pd.DataFrame(
        allvars.h.groupby(level=['person_id','day']).aggregate(quicksum).rename('thesum'))
conDF.index.names = ['id', 'day']
conDF = conDF.merge(peoplehoursperday, left_index=True, right_index=True, how='left')
for it in conDF.itertuples():
    m.addConstr(it.thesum <= 2*it.hours, name=lpnamer('PersonHoursPerDay',it.Index))
m.update()
print(m.NumConstrs)


1271


In [26]:
conDF = pd.DataFrame(allvars.h.groupby(level=['site_id','day']).aggregate(quicksum).rename('thesum'))
conDF.index.names = ['id', 'day']
conDF = conDF.merge(sitehoursperday, left_index=True, right_index=True, how='left')
for it in conDF.itertuples():
    m.addConstr(it.thesum <= 2*it.hours, name=lpnamer('SiteHoursPerDay',it.Index))
m.update()
print(m.NumConstrs)


2975


In [27]:

conDF = pd.DataFrame(allvars.h.groupby(level='person_id').aggregate(quicksum).rename('thesum'))
conDF.index.names = ['id']
conDF = conDF.merge(people, left_index=True, right_index=True, how='left')
for it in conDF.itertuples():
    m.addConstr(it.thesum <= 2*it.total_hrs_per_week, name=lpnamer('TotalHoursPerWeek',it.Index))
m.update()
print(m.NumConstrs)



3312


In [28]:
for it in allvars.itertuples() :
    m.addConstr(it.h <= 2*it.daily_hours*it.pairday, name=lpnamer("PairDayUp",it.Index))
    m.addConstr(2*it.pairday <= it.h, name=lpnamer("PairDayLow",it.Index)) # One hour minimum
m.update()
print(m.NumConstrs)

178554


In [29]:

conDF = pd.DataFrame(allvars.pairday.groupby(level='person_id').aggregate(quicksum).rename('thesum'))
conDF.index.names = ['id']
conDF = conDF.merge(people, left_index=True, right_index=True, how='left')
for it in conDF.itertuples():
    m.addConstr(it.thesum <= it.days_per_week, name=lpnamer('TotalDaysPerWeek[',it.Index))
m.update()
print(m.NumConstrs)

178891


In [30]:
agggb = allvars.pairday.groupby(level=['person_id','site_id']).agg(quicksum).rename('sumpairday')
aggzmerge = pd.merge(pd.DataFrame(agggb), pd.DataFrame(zvars), left_index=True, right_index=True, how='inner')
for it in aggzmerge.itertuples():
    m.addConstr(2*it.z <= it.sumpairday, name=lpnamer('ZVarUp',it.Index))
    m.addConstr(it.sumpairday <= it.sumpairday.size()*it.z, name=lpnamer('ZVarLow',it.Index))
m.update()
print(m.NumConstrs)


241745


### Define the shortage constraints, for each site, measure how well we can cover

In [31]:
shortageSums = pd.DataFrame(allvars.h.groupby(level='site_id').agg(quicksum).rename('thesum'))
shortageSums.index.names = ['id']
conDF = (siteSlackVars
         .merge(shortageSums, left_index=True, right_index=True, how='left')
         .merge(sites, left_index=True, right_index=True, how='left')
         )
for it in conDF.itertuples():
    m.addConstr(it.thesum + it.shortage - it.excess == 2*it.num_hours, name=lpnamer('SiteHours',it.Index))
m.update()
print(m.NumConstrs)


242243


In [32]:

m.setParam(GRB.Param.TimeLimit, 300)
m.setParam(GRB.Param.CutPasses, 1)


Changed value of parameter TimeLimit to 300.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter CutPasses to 1
   Prev: -1  Min: -1  Max: 2000000000  Default: -1


In [33]:
m.optimize()

Optimize a model with 242243 rows, 207665 columns and 1027681 nonzeros
Variable types: 996 continuous, 206669 integer (119048 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective 9934
Presolve removed 115651 rows and 87959 columns
Presolve time: 2.93s
Presolved: 126592 rows, 119706 columns, 620126 nonzeros
Variable types: 0 continuous, 119706 integer (69509 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   15224    4.2500020e+03   4.145890e+03   0.000000e+00      5s
   21471    4.2500000e+03   0.000000e+00   0.000000e+00      9s

Root relaxation: objective 4.250000e+03, 21471 iterations, 5.27 seconds
Total elapsed time = 19.13s
Total elapsed time = 32.64s
Total elapsed time = 41.90s
Total elapsed time = 49.83s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Un

In [34]:
m.addConstr(quicksum(siteSlackVars.shortage) <= m.ObjVal+0.25, name='shortageBound')
m.setObjective(quicksum(siteSlackVars.excess))


In [35]:
resetParams()
m.setParam(GRB.Param.TimeLimit, 180)
m.setParam(GRB.Param.Presolve, 0)
m.setParam(GRB.Param.Cuts, 0)

m.optimize()

Reset all parameters to their default values
Changed value of parameter TimeLimit to 180.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Cuts to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Optimize a model with 242244 rows, 207665 columns and 1028179 nonzeros
Variable types: 996 continuous, 206669 integer (119048 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 4e+03]

Loaded MIP start with objective 399

Variable types: 996 continuous, 206669 integer (123834 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    3347    0.0000000e+00   1.451975e+04   0.000000e+00      7s
   16279    0.0000000e+00   0.000000e+00   0.000000e+00      9s

Root relaxation: objective 0.000000e+00, 16279 iterations, 6.27 seconds
Total ela

In [36]:
m.addConstr(quicksum(siteSlackVars.excess) <= m.ObjVal+0.25, name='excessBound')
conDF = zvars.merge(prefs, left_index=True, right_index=True, how='left')
m.setObjective(quicksum(conDF.pref*conDF.z), sense=GRB.MAXIMIZE)


In [37]:
resetParams()
m.setParam(GRB.Param.MIPGap, 0.01) # Set gap to 1%
m.setParam(GRB.Param.Cuts, 0)
m.setParam(GRB.Param.SubMIPNodes, 1500)
m.setParam(GRB.Param.TimeLimit, 180)
m.optimize()

Reset all parameters to their default values
Changed value of parameter MIPGap to 0.01
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001
Changed value of parameter Cuts to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter SubMIPNodes to 1500
   Prev: 500  Min: 0  Max: 2000000000  Default: 500
Changed value of parameter TimeLimit to 180.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 242245 rows, 207665 columns and 1028677 nonzeros
Variable types: 996 continuous, 206669 integer (119048 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 4e+03]
Presolve removed 98730 rows and 79144 columns
Presolve time: 2.60s
Presolved: 143515 rows, 128521 columns, 656730 nonzeros

Loaded MIP start with objective 22153

Variable types: 738 continuous, 127783 integer (69469 binary)

Root simplex log...

Iteration    Objective       Primal 

In [38]:
allvars.head()

pref  daily_hours  \
person_id          site_id            day                      
MPW0000000272UORBF ADK0000000364ICFPT Mo     64          8.0   
                                      Tu     64          8.0   
                                      We     64          8.0   
                                      Th     64          4.0   
                                      Fr     64          4.0   

                                                                                                               h  \
person_id          site_id            day                                                                          
MPW0000000272UORBF ADK0000000364ICFPT Mo    <gurobi.Var h[MPW0000000272UORBF,ADK0000000364ICFPT,Mo] (value 0.0)>   
                                      Tu    <gurobi.Var h[MPW0000000272UORBF,ADK0000000364ICFPT,Tu] (value 0.0)>   
                                      We   <gurobi.Var h[MPW0000000272UORBF,ADK0000000364ICFPT,We] (value -0.0)>   
                                      Th    <gurobi.Var h[MPW0000000272UORBF,ADK0000000364ICFPT,Th] (value 0.0)>   
                                      Fr    <gurobi.Var h[MPW0000000272UORBF,ADK0000000364ICFPT,Fr] (value 0.0)>   

                                                                                                               pairday  
person_id          site_id            day                                                                               
MPW0000000272UORBF ADK0000000364ICFPT Mo    <gurobi.Var pairday[MPW0000000272UORBF,ADK0000000364ICFPT,Mo] (value 0.0)>  
                                      Tu    <gurobi.Var pairday[MPW0000000272UORBF,ADK0000000364ICFPT,Tu] (value 0.0)>  
                                      We    <gurobi.Var pairday[MPW0000000272UORBF,ADK0000000364ICFPT,We] (value 0.0)>  
                                      Th   <gurobi.Var pairday[MPW0000000272UORBF,ADK0000000364ICFPT,Th] (value -0.0)>  
                                      Fr    <gurobi.Var pairday[MPW0000000272UORBF,ADK0000000364ICFPT,Fr] (value 0.0)>

In [41]:
hvals = pd.Series([h.x for h in allvars.h], index=allvars.index)
pairings = hvals[hvals>0]
pd.DataFrame(pairings.head(10),columns=['hours'])

hours
person_id          site_id            day       
ORY0000000196WQTDH ADK0000000364ICFPT Th     4.0
                                      Fr     4.0
QTA0000000146YSVFJ ADK0000000364ICFPT Mo     2.0
                                      Tu    12.0
                                      We    12.0
TWD0000000123BVYIM ADK0000000364ICFPT Mo    12.0
                                      Th    12.0
ADK0000000130ICFPT ADK0000000390ICFPT Mo    16.0
                                      Tu     2.0
BEL0000000235JDGQU ADK0000000390ICFPT Tu    14.0